In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = np.load('Wafer_Map_Datasets.npz')

데이터를 불러옵니다.

In [5]:
X = data['arr_0']
y = data['arr_1']

"arr_0"이 이미지 데이터 "arr_1"이 결함 여부에 대한 데이터 입니다.

In [6]:
X = X.reshape(X.shape[0], 52, 52, 1).astype('float32') / 255

이미지의 채널을 맞추기 위해 reshape(이미지 수, 행, 열, 채널)을 지정한 후 각 값을 255로 나누어줍니다. 딥러닝은 0 ~ 1 사이의 값일 때 학습 및 예측이 잘됩니다. 이미지 픽셀이 0 ~ 255 사이의 값을 가지므로 255로 나누어 줍니다.

In [7]:
X.shape, y.shape

((38015, 52, 52, 1), (38015, 8))

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop

모델링에 필요한 패키지들을 불러옵니다.

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, random_state = 42, test_size = 0.2)

학습 데이터를 80%, 검증 데이터를 20%로 나누어 줍니다. 이 때 층화추출 방식을 활용해 학습 및 검증 데이터 내에 클래스 비중을 맞춰줍니다.

In [130]:
model = Sequential()
model.add(Conv2D(64, kernel_size = (3, 3), input_shape=(52, 52, 1), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation = 'sigmoid'))

CNN을 활용해 네트워크를 구성합니다. 예측 대상인 y값은 총 8개 컬럼을 가지므로 출력층의 노드 수를 8로 설정합니다. 또한 다중 분류(multi label)의 경우 활성화 함수를 sigmoid를 사용합니다.

In [131]:
model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(learning_rate = 0.003),
              metrics = ['accuracy'])

활성화 함수를 sigmoid를 사용했기 때문에 손실 함수 또한 이진 분류 문제에 사용되는 binary_crossentropy를 사용합니다. 최적화 함수는 Adam을 사용하고 이 때 학습률은 0.003으로 지정합니다. 평가 지표는 정확도를 사용하겠습니다.

In [132]:
es = EarlyStopping(monitor = 'val_accuracy', mode=  'max', patience = 5, verbose = 1)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_accuracy', mode = 'max', patience = 5, verbose = 1, save_best_only = True)

과적합이 발생할 경우 더는 학습을 진행할 필요가 없기 때문에 학습 조기 종료(Early Stopping)와 모델 저장(ModelCheckpoint)을 사용합니다. 검증 정확도를 기준으로 5 에포크 내에 max값이 갱신되지 않으면 학습을 종료하고 검증 정확도가 가장 높을 떄의 모델만을 best_model로 저장합니다.

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 100, batch_size = 32, callbacks = [es, mc])

총 100에포크 동안 매 에포크 마다 32개의 배치로 학습을 진행합니다.

In [ ]:
best_model = load_model('best_model.h5')

학습을 통해 도출된 최고 성능 달성 모델을 불러옵니다.

In [ ]:
valid_acc = model.evaluate(X_val)[1]

검증용 데이터에 대해 정확도를 구합니다.

In [ ]:
print(f'검증용 데이터에 대한 정확도 = {valid_acc}')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

학습 및 검증 정확도를 지정합니다.

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Accuracy of Train & Validation', size = 15)
plt.plot(acc, label = 'Train')
plt.plot(val_acc, label = 'Validation')
plt.show()

에포크가 진행되며 변화한 학습 및 검증 데이터의 정확도를 plotting합니다.

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

학습 및 검증 Loss를 지정합니다.

In [ ]:
plt.figure(figsize = (10, 8))
plt.title('Loss of Train & Validation', size = 15)
plt.plot(loss, label = 'Train')
plt.plot(val_loss, label = 'Validation')
plt.show()

에포크가 진행되며 변화한 학습 및 검증 데이터의 Loss를 plotting합니다.